In [1]:
! pip install numpy
! pip install rasterio
! pip install torchgeo
! pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.8/850.8 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 11.2 MB/s eta 0:0

ERROR: Operation cancelled by user
^C


In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')  # Mounts Drive at /content/drive

# Step 2: Define paths
zip_file_drive = '/content/drive/MyDrive/OmdenaTriestePlasticDebris2025/Data/MARIDA.zip'  # Replace with your ZIP file path in Drive
zip_file_colab = '/content/file.zip'  # Destination path in Colab

# Step 3: Copy the ZIP file from Drive to Colab
import shutil
shutil.copy(zip_file_drive, zip_file_colab)
print(f"Copied {zip_file_drive} to {zip_file_colab}")

# Step 4: Unzip the file
import zipfile
extract_dir = '/content/MARIDA'  # Directory where contents will be extracted
with zipfile.ZipFile(zip_file_colab, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Unzipped {zip_file_colab} to {extract_dir}")

# Optional: List extracted files to verify
import os
extracted_files = os.listdir(extract_dir)
print("Extracted files:", extracted_files)

Mounted at /content/drive
Copied /content/drive/MyDrive/OmdenaTriestePlasticDebris2025/Data/MARIDA.zip to /content/file.zip
Unzipped /content/file.zip to /content/MARIDA
Extracted files: ['splits', 'patches', 'shapefiles', 'labels_mapping.txt']


In [2]:
import re
import os
import pandas as pd
import json

def extract_date_tile(filename):
    """Extract date and tile from filename using regex."""
    pattern = r'^(\d{1,2}-\d{1,2}-\d{2})_([A-Z0-9]+)_\d+$'
    match = re.match(pattern, filename)
    if not match:
        raise ValueError(f"Invalid filename format: {filename}")
    return match.groups()  # Returns tuple (date, tile)

def create_marida_df(data_path, mode='train'):
    """Create DataFrame from MARIDA dataset files."""
    # Determine split file based on mode
    split_files = {'train': 'train_X.txt', 'val': 'val_X.txt', 'test': 'test_X.txt'}
    items_list_path = os.path.join(data_path, 'splits', split_files[mode])

    # Read items list
    with open(items_list_path, 'r') as file:
        items = [item.strip() for item in file]

    # Base path for patches
    items_path = os.path.join(data_path, 'patches')

    # Prepare data lists
    data = {
        'image': [],
        'mask': [],
        'confidence': [],
        'date': [],
        'tile': []
    }

    # Process each item
    for item in items:
        tile = "_".join(item.split("_")[:-1])
        tile_path = os.path.join(items_path, f"S2_{tile}")

        # Define file paths
        base_name = f'S2_{item}'
        paths = {
            'image': os.path.join(tile_path, f'{base_name}.tif'),
            'mask': os.path.join(tile_path, f'{base_name}_cl.tif'),
            'confidence': os.path.join(tile_path, f'{base_name}_conf.tif')
        }

        # Check if all files exist
        if all(os.path.exists(p) for p in paths.values()):
            data['image'].append(paths['image'])
            data['mask'].append(paths['mask'])
            data['confidence'].append(paths['confidence'])
            date, tile = extract_date_tile(item)
            data['date'].append(date)
            data['tile'].append(tile)

    return pd.DataFrame(data)

# MARIDA labels dictionary
MARIDA_LABELS = {
    i: label for i, label in enumerate([
        'Marine Debris', 'Dense Sargassum', 'Sparse Sargassum', 'Natural Organic Material',
        'Ship', 'Clouds', 'Marine Water', 'Sediment-Laden Water', 'Foam', 'Turbid Water',
        'Shallow Water', 'Waves', 'Cloud Shadows', 'Wakes', 'Mixed Water'
    ], 1)
}

# Load and process labels mapping
def load_labels_mapping(file_path):
    """Load and convert labels mapping to DataFrame."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(
        data,
        orient='index',
        columns=[MARIDA_LABELS[i] for i in range(1, 16)]
    ).reset_index().rename(columns={'index': 'image_name'})



In [3]:

data_path = '/content/MARIDA'
df = create_marida_df(data_path, 'train')

# Load labels mapping
# Assigns to each example the set of classes corresponding to the pixels in the imag
labels_df = load_labels_mapping(os.path.join(data_path, 'labels_mapping.txt'))
print(labels_df)

                  image_name  Marine Debris  Dense Sargassum  \
0     S2_1-12-19_48MYU_0.tif              0                0   
1     S2_1-12-19_48MYU_1.tif              0                0   
2     S2_1-12-19_48MYU_2.tif              1                0   
3     S2_1-12-19_48MYU_3.tif              1                0   
4     S2_11-1-19_19QDA_0.tif              0                0   
...                      ...            ...              ...   
1376  S2_9-10-17_16PEC_5.tif              0                0   
1377  S2_9-10-17_16PEC_6.tif              0                0   
1378  S2_9-10-17_16PEC_7.tif              0                0   
1379  S2_9-10-17_16PEC_8.tif              0                0   
1380  S2_9-10-17_16PEC_9.tif              0                0   

      Sparse Sargassum  Natural Organic Material  Ship  Clouds  Marine Water  \
0                    0                         0     1       0             1   
1                    0                         0     0       0         

In [18]:
import torch
from torch.utils.data import Dataset
import rasterio
import numpy as np

class MARIDADataset(Dataset):
    """
    Provides images and masks for the MARIDA dataset.

    Initializes from a pandas DataFrame containing absolute paths to images and masks.

    Adds 2 null bands to ensure compatibility between pretrained TorchGeo weights (13 bands) and MARIDA imagery.

    Handles NaN values by resetting them to 0.

    Transforms MARIDA labels (15 classes) into binary labels: 1 for debris and 0 for no debris.

    Applies z normalization to images

    """
    def __init__(self, df, means, stds, transform=None):
        """
        Args :
            df (pd.DataFrame) : dataframe with columns including image and
                                mask's  absolute paths
            means : band-wise mean computed on MARIDA
            stds : band-wise standard deviation computed on MARIDA
        """
        self.df = df
        self.means = torch.tensor(means, dtype=torch.float32)  # 11 bands
        self.stds = torch.tensor(stds, dtype=torch.float32)    # 11 bands
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Load image (11 bands from MARIDA, already in reflectance range)
        with rasterio.open(self.df.iloc[idx]['image']) as src:
            image = src.read().astype(np.float32)

        # Load mask
        with rasterio.open(self.df.iloc[idx]['mask']) as src:
            mask = src.read(1).astype(np.int64) # Read mask as int64
            mask = (mask == 1).astype(np.int64)  # Set class 1 to 1, all else to 0

        # Convert to torch tensors
        #image = torch.tensor(image)  # Shape: [11, H, W]
        image = torch.tensor(np.nan_to_num(image, nan=0.0))  # Replace NaNs
        mask = torch.tensor(mask)    # Shape: [H, W]

        # Normalize the 11 bands
        image = (image - self.means[:, None, None]) / self.stds[:, None, None]

        # Expand to 13 bands for TorchGeo pretrained model compatibility
        full_image = torch.zeros(13, image.shape[1], image.shape[2])
        band_mapping = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11]  # Skipping B9, B10
        full_image[band_mapping] = image
        if self.transform:
          # Apply any transforms (e.g., resizing, normalization)
          full_image = self.transform(full_image)
          mask = self.transform(mask)
        return full_image, mask



In [5]:
import numpy as np
import rasterio

def compute_marida_stats(df, num_bands=11):
    """
    Compute per-band mean and std from MARIDA GeoTIFFs, handling NaN values.

    Args:
        df (pd.DataFrame): DataFrame with 'image' column containing GeoTIFF paths.
        num_bands (int): Number of bands in each image (11 for MARIDA).

    Returns:
        means (np.ndarray): Mean for each band (NaN-safe).
        stds (np.ndarray): Standard deviation for each band (NaN-safe).
    """
    # Initialize accumulators
    band_sums = np.zeros(num_bands, dtype=np.float64)
    band_sumsq = np.zeros(num_bands, dtype=np.float64)  # For sum of squares
    total_valid_pixels_per_band = np.zeros(num_bands, dtype=np.int64)

    # Process each image
    for img_path in df['image']:
        with rasterio.open(img_path) as src:
            image = src.read().astype(np.float32)
            assert image.shape[0] == num_bands, f"Expected {num_bands} bands, got {image.shape[0]}"

            # Mask NaN and invalid values (e.g., <= 0 could be no-data)
            valid_mask = np.logical_and(np.isfinite(image), image > 0)  # True where valid
            image[~valid_mask] = 0  # Set invalid to 0 for sum calculations (won’t affect valid sums)

            # Sum and sum of squares per band, only for valid pixels
            band_sums += np.sum(image * valid_mask, axis=(1, 2))  # Multiply by mask to exclude invalid
            band_sumsq += np.sum((image ** 2) * valid_mask, axis=(1, 2))
            total_valid_pixels_per_band += np.sum(valid_mask, axis=(1, 2))

    # Compute mean and std (handle case where no valid pixels exist)
    means = np.where(total_valid_pixels_per_band > 0,
                     band_sums / total_valid_pixels_per_band,
                     np.nan)  # NaN if no valid pixels
    variances = np.where(total_valid_pixels_per_band > 0,
                         band_sumsq / total_valid_pixels_per_band - (means ** 2),
                         np.nan)
    stds = np.sqrt(np.maximum(variances, 0))  # Ensure no negative variance due to numerical error

    return means, stds

# Example usage
means_11_bands, stds_11_bands = compute_marida_stats(df[:100])
print("Means:", means_11_bands)
print("Stds:", stds_11_bands)

# Save to file
np.save('means_11_bands.npy', means_11_bands)
np.save('stds_11_bands.npy', stds_11_bands)

Means: [0.03575588 0.03161822 0.02307298 0.0145986  0.01358301 0.01731202
 0.01975166 0.01753542 0.02012366 0.0110173  0.00692523]
Stds: [0.01944577 0.01982479 0.02202    0.02190828 0.02336899 0.04243802
 0.05247064 0.04990999 0.05852649 0.03215673 0.01994174]


In [10]:
import torch.nn.functional as F
from torch import nn
class WeightedDiceLoss(nn.Module):
    def __init__(self, weights=None, smooth=1e-6):
        """
        Weighted Dice Loss for multi-class segmentation.

        Args:
            weights (list or torch.Tensor): Per-class weights (length = num_classes)
            smooth (float): Smoothing factor to avoid division by zero
        """
        super().__init__()
        self.smooth = smooth
        self.weights = torch.tensor(weights, dtype=torch.float32) if weights is not None else None

    def forward(self, inputs, targets):
        preds = F.softmax(inputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=preds.shape[1]).permute(0, 3, 1, 2).float()

        weights = self.weights.to(inputs.device) if self.weights is not None else torch.ones(preds.shape[1], device=inputs.device)

        intersection = (preds * targets_one_hot).sum(dim=(2, 3))
        union = preds.sum(dim=(2, 3)) + targets_one_hot.sum(dim=(2, 3))

        dice_score = (2. * intersection + self.smooth) / (union + self.smooth)
        weighted_dice = weights * dice_score
        return 1 - weighted_dice.mean()

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import rasterio
import numpy as np
import segmentation_models_pytorch as smp
from torchgeo.models import ResNet18_Weights
import os



# DataLoader
dataset = MARIDADataset(df, means_11_bands, stds_11_bands)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

# Load pretrained weights from torchgeo
weights = ResNet18_Weights.SENTINEL2_ALL_MOCO  # Pretrained on Sentinel-2 imagery
in_channels = weights.meta["in_chans"]  # Number of input channels (e.g., 13 for Sentinel-2)

# Set up UNet++ model with pretrained ResNet18 backbone
model = smp.UnetPlusPlus(
    encoder_name="resnet18",            # Backbone architecture
    encoder_weights=None,               # We'll load torchgeo weights manually
    in_channels=in_channels,            # Match Sentinel-2 bands
    classes=2,                          # e.g., debris vs. background
    decoder_use_batchnorm=True          # Optional: improves training stability
)

# Load pretrained weights into the encoder
pretrained_dict = weights.get_state_dict(progress=True)
model_dict = model.encoder.state_dict()
# Filter compatible weights (strict=False skips mismatches)
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.encoder.load_state_dict(model_dict)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
# Define class weights (example: higher weight for Marine Debris, class 1)
class_weights = np.array([1.0, 3.0])/4.  # Emphasize class 1 (Marine Debris)


# Loss and optimizer
criterion = WeightedDiceLoss(weights=class_weights)  # Use weighted Dice Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, masks) in enumerate(dataloader):
        images, masks = images.to(device), masks.to(device)
        # Forward pass
        outputs = model(images)  # Shape: (batch, classes, height, width
        loss = criterion(outputs, masks)
        # Check for NaN or Inf and log only if detected
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Epoch {epoch+1}, Batch {i}: NaN/Inf loss detected: {loss.item()}")
            if torch.any(torch.isnan(images)) or torch.any(torch.isinf(images)):
                print(f"  - Images contain NaN: {torch.any(torch.isnan(images))}, Inf: {torch.any(torch.isinf(images))}")
            if torch.any(torch.isnan(outputs)) or torch.any(torch.isinf(outputs)):
                print(f"  - Outputs contain NaN: {torch.any(torch.isnan(outputs))}, Inf: {torch.any(torch.isinf(outputs))}")
            break  # Stop if NaN/Inf is found
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

# Save the trained model
torch.save(model.state_dict(), "unetpp_marida.pth")
print("Training complete. Model saved as 'unetpp_marida.pth'.")

Epoch [1/10], Loss: 0.8801
Epoch [2/10], Loss: 0.8752
Epoch [3/10], Loss: 0.8723
Epoch [4/10], Loss: 0.8696
Epoch [5/10], Loss: 0.8747
Epoch [6/10], Loss: 0.8750
Epoch [7/10], Loss: 0.7382
Epoch [8/10], Loss: 0.6024
Epoch [9/10], Loss: 0.6029
Epoch [10/10], Loss: 0.6029
Training complete. Model saved as 'unetpp_marida.pth'.
